In [10]:
import pandas as pd
import numpy as pn
from random import randrange

In [26]:
def gini_index_numerical(above_split, below_split, attribute):
    #UPDATING NOW INPUTS ARE DATAFRAMES
    total_above = len(above_split)
    total_below = len(below_split)
    total = total_above + total_below
    #for each class calculate the individual index
    above_index = 0
    below_index = 0
    #probability point is above
    #TODO OPTIMIZE
    labels = ['0','1']
    above_count = above_count['class'].value_counts()
    below_count = below_count['class'].value_counts()
    for i in range(0,len(labels)):
        p_above = above_count[i] / total_above
        p_below = below_count[i] / total_below
        above_index += (p_above * p_above)
        below_index += (p_below * p_below)
    above_index = 1 - above_index
    below_index = 1 - below_index
    return (above_index * (total_above / total)) + (below_index * (total_below / total))

gini_index_numerical takes in two n-d numpy arrays where n represents the number of potential classes. The above_split array represents all of the points located numerically above the split point and the below_split array represents the points with a lesser value.

In [19]:
class Node:
    def __init__(self, splitting_attribute, splitting_point):
        self.splitting_attribute = splitting_attribute
        self.splitting_point = splitting_point
        #TODO change to k for k classes?
        self.children = np.zeros(2)
    
    def new_child_node(self, child_index, child):
        self.children[child_index] = child        
    
    def predict(self, point):
        val = point[self.splitting_attribute]
        if val >= splitting_point:
            return self.children[0]
        else:
            return self.children[1]

The Node class represents each node within the tree. The splitting attribute and point determines the axis and value the node splits on. The above and below values represent the next node or label based on the splitting point.

In [24]:
def split(value, attribute, examples):
    #split where item in row > or < value
    above_values = examples.loc[examples[attribute] >= value]
    below_values = examples.loc[examples[attribute] < value]
    return above_values, below_values
    

In [25]:
def find_best_split(examples, calculate_purity):
    #given a dataframe -> examples find the best splitting value within based
    #on the given purity calculation
    #for row in examples
        #for attribute in row
            #value = row[attribute]
            #split = split(value, examples)
            #purity = calculate_purity(split)
            #if purity better than best purity is best
    #TODO OPTIMIZE THIS
    attributes = ['0','1']
    best_purity = 2
    best_node = None
    best_splits = []
    for index, row in examples.iterrows():
        for val in attributes:
            #split dataframe into 2 dataframes
            above_split, below_split = split(row[val], val, examples)
            purity = calculate_purity(above_split, below_split, val)
            if purity < best_purity:
                best_purity = purity
                #create new node based on best attribute and value
                best_node = Node(val, row[val])
                best_splits = [above_split, below_split]
    return best_node
        

In [17]:
def most_common_class(examples):
    classes = examples['class'].value_counts()
    return classes.idxmax()

In [27]:
def learn_tree(examples, parent_examples, depth, max_depth, calculate_purity):
    #work with ending conditions, if no more examples left then return most common class in parent examples
    if not examples:
        return most_common_class(parent_examples)
    elif (number_of_classes_in_sample == 1) or (depth >= max_depth):
        return most_common_class(examples)
    else:
        #find the best split possible within the current split points
        #splits[] = [above, below]
        root_node, splits = find_best_split(examples, calculate_purity)
        for i in range(splits):
            #above and below calculations
            #split should consist of a subset of examples
            new_node = learn_tree(split,examples,depth-1,max_depth)
            root_node.new_child(i,new_node)
        return root_node